# Create Meteorite Shapefiles

In [159]:
import os
import pandas as pd
import geopandas as gpd
import shapely

from zipfile import ZipFile

### Create a data frame

In [160]:
# Create a data frame from a CSV file
df = pd.read_csv("./assets/meteorite-data-nasa.csv")

### Wrangle the data

In [161]:
# Get rid of unnecessary columns
keep_cols = [
    "name",
    "mass (g)",
    "fall",
    "year",
    "reclat",
    "reclong"
]
df = df[keep_cols]

In [162]:
# Show percentage of rows that have null reclong or reclat values
(df["reclong"].isna() | df["reclat"].isna()).mean()

np.float64(0.16000962463907603)

In [163]:
# Keep only those rows where reclong nor reclat is null
df = df[(df["reclong"].notna() & df["reclat"].notna())]

In [164]:
# Now the null percentage is 0
(df["reclong"].isna() | df["reclat"].isna()).mean()

np.float64(0.0)

### Create a geo data frame

In [165]:
# Construct a geo data frame
geometry = shapely.points(df.pop("reclong"), df.pop("reclat"))
gdf = gpd.GeoDataFrame(df, geometry = geometry)

In [166]:
# Narrow meteorites down to only those that where witnessed
gdf = gdf[gdf["fall"] == "Fell"]

### Create a Shapefile

In [167]:
# Create a Shapefile from the geo data frame
if not os.path.isdir("./shapefiles"):
    os.mkdir("./shapefiles")
gdf.to_file("./shapefiles/meteorites.shp")

/Users/fosse/Desktop/python/meteorite-geospatial-data-visualization/my-env/lib/python3.13/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [168]:
# Compress the Shapefile parts into a ZIP file, simultaneously removing them
with ZipFile("./shapefiles/meteorites.zip", "w") as zipf:
    for file in os.listdir(os.fsencode("./shapefiles/")):
      filename = os.fsdecode(file)
      if not filename.endswith(".zip"):
        zipf.write("./shapefiles/" + filename)
        os.remove("./shapefiles/" + filename)